# Day 20
## Puzzle 1

In [46]:
import numpy as np
import networkx as nx
from collections import defaultdict
from itertools import combinations
from tqdm import tqdm
import math

In [47]:
input_file = 'input_1.txt'
# input_file = 'test_input_1.txt'

Read the input race map.

In [48]:
with open(file=input_file, mode="r") as file:
    race_map = []

    for line in file:
        race_map.append(list(line.strip()))

    race_map_matrix = np.matrix(race_map)

In [49]:
print(race_map_matrix)

[['#' '#' '#' ... '#' '#' '#']
 ['#' '.' '.' ... '.' '.' '#']
 ['#' '.' '#' ... '#' '.' '#']
 ...
 ['#' '.' '#' ... '#' '.' '#']
 ['#' '.' '.' ... '.' '.' '#']
 ['#' '#' '#' ... '#' '#' '#']]


Connect walkable nodes by edges. Also, we add the possible cheat edges (passages) to a list.

In [50]:
m, n = race_map_matrix.shape
start_node = None
end_node = None
edges = []
cheat_edges = set()

for i in range(m):
    for j in range(n):
        current_node_value = race_map_matrix[i, j]

        if current_node_value == 'S':
            start_node = (i, j)

        elif current_node_value == 'E':
            end_node = (i, j)

        if current_node_value != '#':
            if j + 1 < m:
                current_east_node_value = race_map_matrix[i, j + 1]

                if current_east_node_value != '#':
                    edges.append(((i, j), (i, j + 1)))

                elif current_east_node_value == '#' and j + 2 < m:
                    current_east_cheat_node_value = race_map_matrix[i, j + 2]

                    if current_east_cheat_node_value != '#' and ((i, j), (i, j + 2)) not in cheat_edges and ((i, j + 2), (i, j)) not in cheat_edges:
                        cheat_edges.add(((i, j), (i, j + 2)))
                
            if j - 1 >= 0:
                current_west_node_value = race_map_matrix[i, j - 1]

                if current_west_node_value != '#':
                    edges.append(((i, j), (i, j - 1)))

                elif current_west_node_value == '#' and j - 2 >= 0:
                    current_west_cheat_node_value = race_map_matrix[i, j - 2]

                    if current_west_cheat_node_value != '#' and ((i, j), (i, j - 2)) not in cheat_edges and ((i, j - 2), (i, j)) not in cheat_edges:
                        cheat_edges.add(((i, j), (i, j - 2)))

            if i + 1 < n:
                current_south_node_value = race_map_matrix[i + 1, j]

                if current_south_node_value != '#':
                    edges.append(((i, j), (i + 1, j)))

                elif current_south_node_value == '#' and i + 2 < n:
                    current_south_cheat_node_value = race_map_matrix[i + 2, j]

                    if current_south_cheat_node_value != '#' and ((i, j), (i + 2, j)) not in cheat_edges and ((i + 2, j), (i, j)) not in cheat_edges:
                        cheat_edges.add(((i, j), (i + 2, j)))

            if i - 1 >= 0:
                current_north_node_value = race_map_matrix[i - 1, j]

                if current_north_node_value != '#':
                    edges.append(((i, j), (i - 1, j)))

                elif current_north_node_value == '#' and i - 2 >= 0:
                    current_north_cheat_node_value = race_map_matrix[i - 2, j]

                    if current_north_cheat_node_value != '#' and ((i, j), (i - 2, j)) not in cheat_edges and ((i - 2, j), (i, j)) not in cheat_edges:
                        cheat_edges.add(((i, j), (i - 2, j)))

cheat_edges = list(cheat_edges)

Create a graph from the edges (not the cheat edges).

In [51]:
G = nx.Graph()
G.add_edges_from(edges)

Calculate the race length (even though we won't use the result).

In [52]:
race_length = nx.shortest_path_length(G, source=start_node, target=end_node)
race_length

Calculate the "regular" path length between the nodes of the cheat edge and subtract 2 (the length of the cheat path).

In [53]:
number_of_ok_cheats = 0
saved_time_counter = defaultdict(int)

for cheat_edge in cheat_edges:
    saved_time = nx.shortest_path_length(G, source=cheat_edge[0], target=cheat_edge[1]) - 2
    saved_time_counter[saved_time] += 1
    
    if saved_time >= 100:
        number_of_ok_cheats += 1

In [54]:
sorted(saved_time_counter.items())[:15]

[(2, 959),
 (4, 959),
 (6, 257),
 (8, 420),
 (10, 193),
 (12, 293),
 (14, 113),
 (16, 194),
 (18, 80),
 (20, 165),
 (22, 85),
 (24, 149),
 (26, 72),
 (28, 119),
 (30, 54)]

In [55]:
number_of_ok_cheats

1360

## Puzzle 2

Now, we calculate the length of the cheat path using the Manhattan norm, between all pairs of nodes on the regular path. If the cheat path between a pair of nodes is at most 20 picoseconds (and at least 2 picoseconds) and shorter than the regular path between the nodes, it is a valid cheat path.

PS. I know this code can be optimized further... Now it takes about 45 minutes to run.

In [56]:
number_of_ok_cheats_puzzle_2 = 0
saved_time_counter_puzzle_2 = defaultdict(int)

for n1, n2 in tqdm(combinations(G.nodes, 2), total=math.comb(len(G.nodes), 2)):
    manhattan_distance = int(np.linalg.norm(x=np.array(n1) - np.array(n2), ord=1))

    if 2 <= manhattan_distance <= 20:
        race_map_shortest_path = nx.shortest_path_length(G, source=n1, target=n2)
        saved_time = race_map_shortest_path - manhattan_distance

        if saved_time > 0:
            saved_time_counter_puzzle_2[saved_time] += 1

            if saved_time >= 100:
                number_of_ok_cheats_puzzle_2 += 1

100%|██████████| 44297578/44297578 [42:03<00:00, 17556.34it/s] 


In [57]:
sorted(saved_time_counter_puzzle_2.items())[:15]

[(2, 16817),
 (4, 44511),
 (6, 14768),
 (8, 32978),
 (10, 13358),
 (12, 27690),
 (14, 12202),
 (16, 23810),
 (18, 11229),
 (20, 20320),
 (22, 10486),
 (24, 18146),
 (26, 9810),
 (28, 18031),
 (30, 9384)]

In [58]:
number_of_ok_cheats_puzzle_2

1005476